# Genetic Risk Prediction Analysis

This notebook demonstrates genetic risk prediction using machine learning on OpenShift AI.

## Overview
- Load and analyze genetic sequence data
- Process VEP annotations
- Build ML models for risk prediction
- Demonstrate real-time processing with Kafka
- Monitor scaling and cost attribution

Based on research from PMC7613081: Machine learning approaches for genetic risk prediction

In [ ]:
# Install required packages
import sys
!{sys.executable} -m pip install kafka-python pandas numpy scikit-learn matplotlib seaborn biopython requests

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from kafka import KafkaProducer, KafkaConsumer
import json
import time
import requests
from Bio.Seq import Seq
from Bio.SeqUtils import GC
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🧬 BioPython available")
print(f"📈 Scikit-learn available")

## 1. Generate Sample Genetic Data

Create synthetic genetic sequences and risk factors for demonstration

In [ ]:
# Generate sample genetic data
np.random.seed(42)

def generate_genetic_sequence(length=100):
    """Generate a random DNA sequence"""
    bases = ['A', 'T', 'G', 'C']
    return ''.join(np.random.choice(bases, length))

def calculate_genetic_features(sequence):
    """Calculate features from genetic sequence"""
    seq_obj = Seq(sequence)
    return {
        'length': len(sequence),
        'gc_content': GC(sequence),
        'a_count': sequence.count('A'),
        't_count': sequence.count('T'),
        'g_count': sequence.count('G'),
        'c_count': sequence.count('C'),
        'at_ratio': (sequence.count('A') + sequence.count('T')) / len(sequence),
        'purine_ratio': (sequence.count('A') + sequence.count('G')) / len(sequence)
    }

# Generate sample dataset
n_samples = 1000
genetic_data = []

for i in range(n_samples):
    # Generate sequence
    sequence = generate_genetic_sequence(np.random.randint(80, 120))
    features = calculate_genetic_features(sequence)
    
    # Simulate risk factors (based on GC content and sequence patterns)
    risk_score = (
        features['gc_content'] * 0.3 +
        features['at_ratio'] * 0.2 +
        np.random.normal(0, 10)
    )
    
    # Binary risk classification
    high_risk = 1 if risk_score > 25 else 0
    
    genetic_data.append({
        'sample_id': f'SAMPLE_{i:04d}',
        'sequence': sequence,
        'risk_score': risk_score,
        'high_risk': high_risk,
        **features
    })

# Create DataFrame
df = pd.DataFrame(genetic_data)
print(f"✅ Generated {len(df)} genetic samples")
print(f"📊 High risk samples: {df['high_risk'].sum()} ({df['high_risk'].mean()*100:.1f}%)")
df.head()

## 2. Exploratory Data Analysis

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# GC Content distribution
axes[0,0].hist(df['gc_content'], bins=30, alpha=0.7, color='skyblue')
axes[0,0].set_title('GC Content Distribution')
axes[0,0].set_xlabel('GC Content (%)')
axes[0,0].set_ylabel('Frequency')

# Risk score by GC content
scatter = axes[0,1].scatter(df['gc_content'], df['risk_score'], 
                           c=df['high_risk'], cmap='RdYlBu', alpha=0.6)
axes[0,1].set_title('Risk Score vs GC Content')
axes[0,1].set_xlabel('GC Content (%)')
axes[0,1].set_ylabel('Risk Score')
plt.colorbar(scatter, ax=axes[0,1], label='High Risk')

# Sequence length distribution
axes[1,0].hist(df['length'], bins=20, alpha=0.7, color='lightgreen')
axes[1,0].set_title('Sequence Length Distribution')
axes[1,0].set_xlabel('Sequence Length')
axes[1,0].set_ylabel('Frequency')

# Risk distribution
risk_counts = df['high_risk'].value_counts()
axes[1,1].pie(risk_counts.values, labels=['Low Risk', 'High Risk'], 
              autopct='%1.1f%%', colors=['lightblue', 'salmon'])
axes[1,1].set_title('Risk Distribution')

plt.tight_layout()
plt.show()

# Summary statistics
print("\n📊 Summary Statistics:")
print(df[['gc_content', 'length', 'risk_score', 'at_ratio', 'purine_ratio']].describe())

## 3. Machine Learning Model Training

In [ ]:
# Prepare features for ML
feature_columns = ['gc_content', 'length', 'at_ratio', 'purine_ratio', 'a_count', 't_count', 'g_count', 'c_count']
X = df[feature_columns]
y = df['high_risk']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"📊 Training set: {len(X_train)} samples")
print(f"📊 Test set: {len(X_test)} samples")
print(f"📊 Feature columns: {feature_columns}")

In [ ]:
# Train Random Forest model
print("🤖 Training Random Forest model...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("✅ Model training completed!")

# Evaluate model
print("\n📈 Model Performance:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n🎯 Feature Importance:")
print(feature_importance)

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0])
axes[0].set_title('Confusion Matrix')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('Actual')

# Feature Importance
axes[1].barh(feature_importance['feature'], feature_importance['importance'])
axes[1].set_title('Feature Importance')
axes[1].set_xlabel('Importance')

plt.tight_layout()
plt.show()

# Model accuracy
accuracy = rf_model.score(X_test, y_test)
print(f"\n🎯 Model Accuracy: {accuracy:.3f}")